In [25]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import time
import csv
import json
import re

## Authentication

In [26]:
CLIENT_ID = 'd8415d5149674a91a7e81de017c789d7'
CLIENT_SECRET = '576b0598e8014aeb93f7bc1ab69ac410'

client_credentials_manager = SpotifyClientCredentials(CLIENT_ID, CLIENT_SECRET)
spotify = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

## Playlists

In [27]:
PARAMETERS_FILE = '../parameters.json'

with open(PARAMETERS_FILE) as f:
    parameters = json.load(f)

pattern = r'/playlist/([^/?]+)'

playlist_id = list()

for url in parameters['playlist_id']:
    match = re.search(pattern, url)

    if match:
        playlist_id.append(match.group(1))

playlist_id

['3tTo8cKpIQbLJ3MTzgEVjV', '1Kzd8jonR1022i8dL8eDda']

### Mainstream Genres

## Get Artist Data

In [28]:
def get_artist_genres(artist_id:str) -> list:
    artist = spotify.artist(artist_id)

    return artist['genres']


def get_related_artists_names(artist_id:str) -> list:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_names = [related_artist['name'] for related_artist in related_artists['artists']]

    return related_artists_names

def get_related_artists_genres(artist_id:str, track_genres:list) -> set:
    related_artists = spotify.artist_related_artists(artist_id)

    related_artists_genres = set()

    for related_artist in related_artists['artists']:
        genres = set(related_artist['genres'])

        for genre in genres:
            related_artists_genres.add(genre)

        print(related_artists_genres)

    return list(related_artists_genres - set(track_genres))

def get_generic_genres(track_genres:list, track_related_genres:list) -> list:
    generic_genres = list()

    for track_genre in track_genres:
        for genre in parameters['generic_genres']:
            if genre in track_genre:
                generic_genres.append(genre)

    for track_related_genre in track_related_genres:
        for genre in parameters['generic_genres']:
            if genre in track_related_genre:
                generic_genres.append(genre)

    return generic_genres

## Get Track Data

In [29]:
def get_tracks_id(playlist_id:str) -> list:  
    tracks_id = list()
    playlist_items = spotify.playlist_items(playlist_id)

    for item in playlist_items['items']:
        track = item['track']
        tracks_id.append(track['id'])

    return tracks_id

def get_track_data(track_id:str) -> dict:
    metadata = spotify.track(track_id)
    audio_features = spotify.audio_features(track_id)

    artist_id = metadata['album']['artists'][0]['id']
    track_genres = get_artist_genres(artist_id)
    related_track_genres = get_related_artists_genres(artist_id, track_genres)


    return {
        'id': metadata['id'],
        'name': metadata['name'],
        'album': metadata['album']['name'],
        'artist': metadata['album']['artists'][0]['name'],
        'genres': track_genres,
        'related_artists': get_related_artists_names(artist_id),
        'related_genres': related_track_genres,
        'generic_genres': get_generic_genres(track_genres, related_track_genres),
        'release_date': metadata['album']['release_date'],
        'duration_ms': metadata['duration_ms'],
        'popularity': metadata['popularity'],
        'acousticness': audio_features[0]['acousticness'],
        'danceability': audio_features[0]['danceability'],

        'energy': audio_features[0]['energy'],
        'instrumentalness': audio_features[0]['instrumentalness'],
        'key': audio_features[0]['key'],
        'liveness': audio_features[0]['liveness'],
        'loudness': audio_features[0]['loudness'],
        'mode': audio_features[0]['mode'],
        'speechiness': audio_features[0]['speechiness'],
        'tempo': audio_features[0]['tempo'],
        'time_signature': audio_features[0]['time_signature'],
        'valence': audio_features[0]['valence'],
    }

In [30]:
data = dict()

for _id in playlist_id:
    tracks_id = get_tracks_id(_id)
    tracks_data = list()

    for track_id in tracks_id:
        time.sleep(.5)
        tracks_data.append(get_track_data(track_id))
        
    data[_id] = tracks_data

{'singer-songwriter', 'blues rock', 'album rock', 'folk rock', 'hard rock', 'rock', 'psychedelic rock', 'supergroup', 'british blues', 'classic rock'}
{'singer-songwriter', 'blues rock', 'album rock', 'folk rock', 'hard rock', 'rock', 'country rock', 'psychedelic rock', 'supergroup', 'british blues', 'classic rock', 'british invasion'}
{'singer-songwriter', 'blues rock', 'album rock', 'folk rock', 'hard rock', 'rock', 'country rock', 'psychedelic rock', 'supergroup', 'electric blues', 'british blues', 'classic rock', 'blues', 'british invasion'}
{'singer-songwriter', 'heartland rock', 'blues rock', 'album rock', 'folk rock', 'hard rock', 'rock', 'country rock', 'psychedelic rock', 'supergroup', 'mellow gold', 'electric blues', 'british blues', 'classic rock', 'blues', 'british invasion'}
{'album rock', 'soft rock', 'classic canadian rock', 'mellow gold', 'electric blues', 'classic rock', 'british invasion', 'singer-songwriter', 'heartland rock', 'country rock', 'acid rock', 'proto-meta

In [31]:
for key,value in data.items():
    print(key, value)
    with open(f"../data/metadata/{key}.json", "w") as f:
        json.dump(value, f, indent=4)

3tTo8cKpIQbLJ3MTzgEVjV [{'id': '4sQdW03IQh3tsZeBELdt5G', 'name': 'Foxey Lady', 'album': 'Are You Experienced', 'artist': 'Jimi Hendrix', 'genres': ['acid rock', 'album rock', 'classic rock', 'hard rock', 'proto-metal', 'psychedelic rock', 'rock'], 'related_artists': ['Cream', 'The Yardbirds', 'Canned Heat', 'Derek & The Dominos', 'Steppenwolf', 'Janis Joplin', 'Ten Years After', 'Free', 'Jefferson Airplane', 'The Doors', 'The Animals', 'Big Brother & The Holding Company', 'Stevie Ray Vaughan', 'Rory Gallagher', 'Jim Morrison', 'Traffic', 'John Mayall & The Bluesbreakers', 'Allman Brothers Band', 'Albert King', 'The Who'], 'related_genres': ['soft rock', 'classic canadian rock', 'roots rock', 'mellow gold', 'electric blues', 'progressive rock', 'southern rock', 'british invasion', 'singer-songwriter', 'heartland rock', 'soul blues', 'country rock', 'instrumental rock', 'memphis soul', 'british blues', 'jam band', 'art rock', 'symphonic rock', 'blues rock', 'folk rock', 'texas blues', 'b

In [32]:
data

{'3tTo8cKpIQbLJ3MTzgEVjV': [{'id': '4sQdW03IQh3tsZeBELdt5G',
   'name': 'Foxey Lady',
   'album': 'Are You Experienced',
   'artist': 'Jimi Hendrix',
   'genres': ['acid rock',
    'album rock',
    'classic rock',
    'hard rock',
    'proto-metal',
    'psychedelic rock',
    'rock'],
   'related_artists': ['Cream',
    'The Yardbirds',
    'Canned Heat',
    'Derek & The Dominos',
    'Steppenwolf',
    'Janis Joplin',
    'Ten Years After',
    'Free',
    'Jefferson Airplane',
    'The Doors',
    'The Animals',
    'Big Brother & The Holding Company',
    'Stevie Ray Vaughan',
    'Rory Gallagher',
    'Jim Morrison',
    'Traffic',
    'John Mayall & The Bluesbreakers',
    'Allman Brothers Band',
    'Albert King',
    'The Who'],
   'related_genres': ['soft rock',
    'classic canadian rock',
    'roots rock',
    'mellow gold',
    'electric blues',
    'progressive rock',
    'southern rock',
    'british invasion',
    'singer-songwriter',
    'heartland rock',
    'soul bl